In [67]:
import os
import json
import pandas as pd
from glob import glob
from tqdm import tqdm
from collections import defaultdict

In [68]:
# Configuration - You can modify these variables
data_dir = r"C:\Users\weiha\Desktop\benchmark_result"  # Base directory containing results
pipeline_list = ["supervised"]  # List of pipelines to analyze
task_name_list =["MotionSourceRecognition","BreathingDetection_Subset","Localization","FallDetection","ProximityRecognition","HumanActivityRecognition","HumanIdentification"]
# task_name_list = ["BreathingDetection_Subset"]  # List of tasks to analyze
model_name_list = ["mlp", "lstm", "resnet18", "transformer", "vit", "patchtst", "timesformer1d"] # List of models to analyze


# Initialize an empty list to store results
results_data = []

# Iterate through all combinations
for pipeline in pipeline_list:
    for task_name in task_name_list:
        for model_name in model_name_list:
            # Find all experiment folders for the current combination
            exp_pattern = os.path.join(data_dir, pipeline, task_name, model_name, "params_*")
            exp_folders = glob(exp_pattern)
            
            print(f"Found {len(exp_folders)} experiments for {pipeline}/{task_name}/{model_name}")
            
            # Process each experiment
            for exp_folder in tqdm(exp_folders, desc=f"Processing {pipeline}/{task_name}/{model_name}"):
                # Extract experiment ID
                exp_id = os.path.basename(exp_folder)
                
                # Define paths for config and results files
                config_filename = f"{model_name}_{task_name}_config.json"
                results_filename = f"{model_name}_{task_name}_results.json"
                
                config_path = os.path.join(exp_folder, config_filename)
                results_path = os.path.join(exp_folder, results_filename)
                
                # Skip if either file doesn't exist
                if not os.path.exists(config_path) or not os.path.exists(results_path):
                    print(f"Missing files for {exp_folder}, skipping")
                    continue
                
                # Read config file
                try:
                    with open(config_path, 'r') as f:
                        config_data = json.load(f)
                    
                    # Extract required fields from config
                    learning_rate = config_data.get('learning_rate')
                    weight_decay = config_data.get('weight_decay')
                    seed = config_data.get('seed')
                    batch_size = config_data.get('batch_size')
                    
                    # Skip if seed is not available (old version)
                    if seed is None:
                        print(f"Seed not available in {exp_folder}, skipping")
                        continue
                        
                except Exception as e:
                    print(f"Error reading config file {config_path}: {e}")
                    continue
                
                # Read results file
                try:
                    with open(results_path, 'r') as f:
                        results_data_json = json.load(f)
                except Exception as e:
                    print(f"Error reading results file {results_path}: {e}")
                    continue
                
                # Create a dictionary for the current experiment
                exp_result = {
                    'pipeline': pipeline,
                    'task_name': task_name,
                    'model_name': model_name,
                    'experiment_id': exp_id,
                    'learning_rate': learning_rate,
                    'weight_decay': weight_decay,
                    'batch_size':batch_size,
                    'seed': seed,
                }
                
                # Extract test results for all test sets
                for test_name, test_results in results_data_json.items():
                    if isinstance(test_results, dict):
                        for metric_name, metric_value in test_results.items():
                            column_name = f"{test_name}_{metric_name}"
                            exp_result[column_name] = metric_value
                
                # Add to results list
                results_data.append(exp_result)


results_df = pd.DataFrame(results_data)


for task in task_name_list:
    for model in model_name_list:
        task_model_df = results_df[(results_df['task_name'] == task) & (results_df['model_name'] == model)]
        if not task_model_df.empty:
            print(f"\n{task} - {model} (count: {len(task_model_df)})")
            
            # Find metrics columns
            metric_columns = [col for col in task_model_df.columns if any(col.endswith(m) for m in ['_loss', '_accuracy', '_f1'])]
            
            if metric_columns:
                avg_metrics = task_model_df[metric_columns].mean()
                for metric, value in avg_metrics.items():
                    print(f"  {metric}: {value:.4f}")

results_df = results_df[results_df["batch_size"] == 128].reset_index(drop=True)



Found 4 experiments for supervised/MotionSourceRecognition/mlp


Processing supervised/MotionSourceRecognition/mlp: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 617.29it/s]


Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\MotionSourceRecognition\mlp\params_d0537c49, skipping
Found 4 experiments for supervised/MotionSourceRecognition/lstm


Processing supervised/MotionSourceRecognition/lstm: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 853.54it/s]


Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\MotionSourceRecognition\lstm\params_c579fb8f, skipping
Found 4 experiments for supervised/MotionSourceRecognition/resnet18


...

Found 0 experiments for supervised/HumanIdentification/patchtst


Processing supervised/HumanIdentification/patchtst: 0it [00:00, ?it/s]


Found 0 experiments for supervised/HumanIdentification/timesformer1d


Processing supervised/HumanIdentification/timesformer1d: 0it [00:00, ?it/s]



MotionSourceRecognition - mlp (count: 3)
  test_hard_loss: 0.0534
  test_hard_accuracy: 0.9791
  test_medium_loss: 0.0281
  test_medium_accuracy: 0.9917
  test_loss: 0.0330
  test_accuracy: 0.9886
  test_easy_loss: 0.0278
  test_easy_accuracy: 0.9862
  test_medium_id_loss: nan
  test_medium_id_accuracy: nan
  test_easy_id_loss: nan
  test_easy_id_accuracy: nan
  test_hard_id_loss: nan
  test_hard_id_accuracy: nan

MotionSourceRecognition - lstm (count: 3)
  test_hard_loss: 0.0966
  test_hard_accuracy: 0.9767
  test_medium_loss: 0.0538
  test_medium_accuracy: 0.9866
  test_loss: 0.0631
  test_accuracy: 0.9842
  test_easy_loss: 0.0633
  test_easy_accuracy: 0.9826
  test_medium_id_loss: nan
  test_medium_id_accuracy: nan
  test_easy_id_loss: nan
  test_easy_id_accuracy: nan
  test_hard_id_loss: nan
  test_hard_id_accuracy: nan

MotionSourceRecognition - resnet18 (count: 3)
  test_hard_loss: 0.0185
  test_hard_accuracy: 0.9948
  test_medium_loss: 0.0127
  test_medium_accuracy: 0.9957
  te

In [69]:
results_df[["task_name","model_name","test_accuracy", "test_f1_score"]]
# results_df[results_df['task_name'] == 'BreathingDetection_Subset']

,task_name,model_name,test_accuracy,test_f1_score
0,MotionSourceRecognition,mlp,0.987999,0.987996
1,MotionSourceRecognition,mlp,0.988466,0.988469
2,MotionSourceRecognition,mlp,0.989401,0.989401
3,MotionSourceRecognition,lstm,0.985973,0.985980
4,MotionSourceRecognition,lstm,0.984414,0.984434
...,...,...,...,...
79,FallDetection,patchtst,0.949254,0.949204
80,FallDetection,timesformer1d,0.944279,0.944166
81,FallDetection,timesformer1d,0.936318,0.936412
82,FallDetection,timesformer1d,0.923383,0.923943


In [70]:
# Save results to CSV
output_path = "all_results_summary.csv"
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")


Results saved to all_results_summary.csv


## Generate Error Bar


In [71]:
def generate_errorbar_summary_with_latex(df: pd.DataFrame,
                                        task_list,
                                        metric_columns,
                                        learning_rate=0.001,
                                        weight_decay=0.00001,
                                        model_order=None,
                                        precision=4):

    # Filter by learning rate and weight decay
    filtered_df = df[(df["learning_rate"] == learning_rate) & 
                     (df["weight_decay"] == weight_decay)]

    # Calculate mean ± std per model per task
    summary = []
    for task in task_list:
        task_df = filtered_df[filtered_df["task_name"] == task]
        for model in sorted(task_df["model_name"].unique()):
            row = {"task": task, "model": model}
            model_df = task_df[task_df["model_name"] == model]
            for col in metric_columns:
                if col in model_df.columns:
                    values = model_df[col].dropna()
                    if len(values) > 0:
                        mean = values.mean()
                        std = values.std(ddof=1)
                        row[col] = f"{mean:.{precision}f}±{std:.{precision}f}"
                    else:
                        row[col] = "--"
                else:
                    row[col] = "--"
            summary.append(row)
    summary_df = pd.DataFrame(summary)

    # Prepare LaTeX data
    latex_data = defaultdict(lambda: defaultdict(lambda: {"accuracy": "--", "f1_score": "--"}))
    for _, row in summary_df.iterrows():
        model = row["model"]
        task = row["task"]
        latex_data[model][task]["accuracy"] = row.get(metric_columns[0], "--")
        latex_data[model][task]["f1_score"] = row.get(metric_columns[1], "--")

    if model_order is None:
        model_order = sorted(summary_df["model"].unique())

    # Generate LaTeX rows
    latex_rows = []
    for model in model_order:
        acc_f1_values = []
        for task in task_list:
            acc = latex_data[model][task]["accuracy"]
            f1 = latex_data[model][task]["f1_score"]
            acc_f1_values.extend([acc, f1])
        row_line = f"{model.upper()} & " + " & ".join(acc_f1_values) + r" \\"
        latex_rows.append(row_line)

    # LaTeX header and footer
    latex_table_header = r"""
\begin{table}[t]
\centering
\caption{Standard supervised evaluation on four core tasks. We report Accuracy and F1-score for each model.}
\label{tab:supervised-results}
\resizebox{\textwidth}{!}{%
\begin{tabular}{
    >{\centering\arraybackslash}p{2.5cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}
}
     \toprule
    Model & \multicolumn{2}{c|}{\textbf{Motion Source Recognition}} & \multicolumn{2}{c|}{\textbf{Fall Detection}} & \multicolumn{2}{c|}{\textbf{Breathing Detection}} & \multicolumn{2}{c}{\textbf{Localization}} \\
    \midrule
"""
    latex_table_footer = r"""
\hline
\end{tabular}%
}
\end{table}
"""
    full_latex_code = latex_table_header + "\n" + "\n".join(latex_rows) + latex_table_footer
    return summary_df, full_latex_code





In [72]:
task_list = ["MotionSourceRecognition", "FallDetection", "BreathingDetection", "Localization"]
metric_columns = ["test_accuracy", "test_f1_score"]
model_order=["mlp", "lstm", "transformer", "resnet18", "patchtst", "timesformer1d"]

summary_df, latex_code = generate_errorbar_summary_with_latex(df=results_df, 
                                                              task_list=task_list,
                                                              metric_columns=metric_columns,
                                                              model_order=model_order,
                                                             precision=4)



# 查看结果表格
display(summary_df)

# 打印 LaTeX 表格代码
print(latex_code)


,task,model,test_accuracy,test_f1_score
0,MotionSourceRecognition,lstm,0.9842±0.0019,0.9842±0.0019
1,MotionSourceRecognition,mlp,0.9886±0.0007,0.9886±0.0007
2,MotionSourceRecognition,patchtst,0.9886±0.0019,0.9886±0.0019
3,MotionSourceRecognition,resnet18,0.9956±0.0007,0.9956±0.0007
4,MotionSourceRecognition,timesformer1d,0.9838±0.0017,0.9839±0.0017
5,MotionSourceRecognition,transformer,0.9861±0.0027,0.9861±0.0027
6,MotionSourceRecognition,vit,0.9898±0.0015,0.9898±0.0015
7,FallDetection,lstm,0.9493±0.0051,0.9492±0.0050
8,FallDetection,mlp,0.9216±0.0091,0.9217±0.0092
9,FallDetection,patchtst,0.9403±0.0074,0.9403±0.0073



\begin{table}[t]
\centering
\caption{Standard supervised evaluation on four core tasks. We report Accuracy and F1-score for each model.}
\label{tab:supervised-results}
\resizebox{\textwidth}{!}{%
\begin{tabular}{
    >{\centering\arraybackslash}p{2.5cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}|
    >{\centering\arraybackslash}p{1.8cm}
}
     \toprule
    Model & \multicolumn{2}{c|}{\textbf{Motion Source Recognition}} & \multicolumn{2}{c|}{\textbf{Fall Detection}} & \multicolumn{2}{c|}{\textbf{Breathing Detection}} & \multicolumn{2}{c}{\textbf{Localization}} \\
    \midrule

MLP & 0.9886±0.0007 & 0.9886±0.0007 & 0.9216±0.0091 & 0.9217±0.0092 & -- & -- & 0.8714±0.0080 & 0.8690±0.0083 \\
LSTM & 0.9842±0.0019 & 0.9842±0.0019 & 0.9493±0.0051

In [73]:
summary_df.to_csv('result_with_error_bar.csv')